In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from os import path

In [3]:
folder_path = path.join('.','data')

In [4]:
doe = pd.read_excel(path.join(folder_path, 'doe_lhsmu_locked.xlsx'),
                    header = 0, usecols = 'A:J')

In [5]:
doe.head()

,batch,layer_thickness,Oscillator,Recoat Speed,roller_speed,roller_rpm,drying speed,drying_power,saturation,binder_set_time
0,1,170.950131,1967.331486,78.906433,4.325408,195.740100,8.975193,57.789017,59.215849,4.682984
1,2,218.833816,2059.599336,61.607811,3.001314,277.268534,16.391061,59.744374,89.594092,9.374743
2,3,129.909640,1937.495407,43.603111,5.855950,169.602720,19.940300,39.618434,80.671603,8.926232
3,4,141.806415,1871.663699,90.385153,17.915776,160.335560,11.752588,71.820288,91.839952,9.035047
4,5,125.986854,2112.279386,52.067637,11.647734,265.095430,17.432956,57.658267,67.618446,5.008873


In [6]:
sample_dimension = pd.read_excel(path.join(folder_path, 'process_map_data.xlsx'),
                                 header = 0, usecols = 'A:E')

In [7]:
sample_dimension.head()

,batch,id,x,y,z
0,1,1,20.47,15.45,10.49
1,1,1,20.41,15.29,10.37
2,1,1,20.68,15.52,10.46
3,1,2,20.37,15.46,10.34
4,1,2,20.31,15.48,10.32


In [8]:
archimedes = pd.read_excel(path.join(folder_path, 'process_map_data_archimedes.xlsx'),
                                 header = 0, usecols = 'A:F')

In [9]:
archimedes.head()

,batch,id,dry weight,immersed weight,soaked weight,run
0,1,4.0,16.0628,14.3834,16.8908,1
1,1,4.0,16.0612,14.3735,16.8770,1
2,1,4.0,16.0562,14.3548,16.8385,1
3,2,2.0,16.7028,14.9509,17.5258,1
4,2,2.0,16.6986,14.9242,17.4925,1


In [10]:

liq_den = 783.8
ss316 = 8000

density = archimedes['dry weight']/(archimedes['soaked weight']-archimedes['immersed weight'])*liq_den/ss316

In [18]:
pd.concat([density, archimedes['batch']], axis=0)

,0,batch
0,0.627643,1
1,0.628558,1
2,0.633372,1
3,0.635542,2
4,0.637015,2
...,...,...
206,0.609000,9
207,0.604540,10
208,0.602098,10
209,0.601610,10


In [15]:
den_result.head()

0    1
1    1
2    1
3    2
4    2
Name: batch, dtype: object

In [ ]:
archimedes['density'] = density

In [ ]:
archimedes = archimedes.join(doe.set_index('batch'), on='batch')

In [ ]:
archimedes.head()

In [ ]:
settings = doe.columns[1:]

In [ ]:
settings

In [ ]:
fig, axs = plt.subplots(3,3, figsize = (10,10))

for ax,param in zip(axs.flatten(),settings):
    sns.scatterplot(x = param, y = 'density', data = archimedes,ax= ax)
    ax.set_title('Density vs {}'.format(param))

plt.tight_layout()
plt.show()

In [ ]:
from itertools import combinations

comb = list(combinations(settings, 2))
n_pair = len(comb)

fig, axs = plt.subplots(int(n_pair**0.5), int(n_pair**0.5), figsize = (50,50))

for ax, param in zip(axs.flatten(),comb):
    sns.scatterplot(x = param[0], y = param[1], hue = 'density', data = archimedes, ax= ax)
    ax.set_title('{} vs {}'.format(param[0], param[1]))

plt.show()

In [ ]:
dimension = pd.read_excel(path.join(folder_path, 'process_map_data.xlsx'),
                                 header = 0, usecols = 'A:E')

In [ ]:
dimension['x'] = abs(dimension['x'] - 20.0)
dimension['y'] = abs(dimension['y'] - 15.0)
dimension['z'] = abs(dimension['z'] - 10.0)

In [ ]:
dimension.head()

In [ ]:
max_agg = dimension.groupby(['batch','id']).max()

In [ ]:
max_agg = max_agg.join(doe.set_index('batch'), on='batch')

In [ ]:
max_agg.head()

In [ ]:
fig, axs = plt.subplots(3,3, figsize = (10,10))

for ax,param in zip(axs.flatten(),settings):
    ax.plot(max_agg[param],max_agg['x'],'.')
    ax.set_title('x vs {}'.format(param))

plt.show()

In [ ]:
fig, axs = plt.subplots(3,3, figsize = (10,10))

for ax,param in zip(axs.flatten(),settings):
    ax.plot(max_agg[param],max_agg['y'],'.')
    ax.set_title('y vs {}'.format(param))

plt.show()

In [ ]:
fig, axs = plt.subplots(3,3, figsize = (10,10))

for ax,param in zip(axs.flatten(),settings):
    ax.plot(max_agg[param],max_agg['z'],'.')
    ax.set_title('z vs {}'.format(param))

plt.show()

In [ ]:
import sklearn.gaussian_process as gp
import optuna
from sklearn.model_selection import cross_val_score

data = archimedes.iloc[:,6:].to_numpy()

In [ ]:
from sklearn import preprocessing

x_min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = x_min_max_scaler.fit_transform(data[:,1:])
y = data[:,0]

In [ ]:
def gp_hyp_tuning(X, y):
    def objective(trial):

        ct = trial.suggest_uniform('constant kernel', 1e-3, 1.0)
        ls = trial.suggest_uniform('length scale', 1e-1, 150)
        nu = trial.suggest_uniform('nu', 1.0, 3.0)
        noise = trial.suggest_uniform('noise', 1e-5, 1.0)
        
        kernel = gp.kernels.ConstantKernel(ct)*gp.kernels.Matern(length_scale=ls, length_scale_bounds=(1e-1, 10000), nu=nu)
        regressor = gp.GaussianProcessRegressor(kernel=kernel, alpha = noise**2, n_restarts_optimizer=50)
        
        cross_val = cross_val_score(regressor, X, y, cv = 10)
        
        trial.set_user_attr('std', np.std(cross_val))
        
        return np.mean(cross_val)
    return objective


hype_tune_func = gp_hyp_tuning(X_scaled, y)

In [ ]:
from sqlalchemy import create_engine

optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(study_name='hype tune gp', direction='maximize', 
                            storage = 'sqlite:///data/hyp_tune.db',
                           load_if_exists=True)



In [ ]:
study.optimize(hype_tune_func, n_trials=100, n_jobs = 1)

In [ ]:
study.best_trial

In [ ]:
study.best_trial.params

In [ ]:
from sklearn.model_selection import train_test_split

def create_regressor(trial):
    param = trial.params
    ct = param['constant kernel']
    ls = param['length scale']
    noise = param['noise']
    nu = param['nu']
    
    kernel = gp.kernels.ConstantKernel(ct)*gp.kernels.Matern(length_scale=ls, length_scale_bounds=(1e-1, 10000), nu=nu)
    regressor = gp.GaussianProcessRegressor(kernel=kernel, alpha = noise**2, n_restarts_optimizer=50)
    
    return regressor

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

optimal_regressor = create_regressor(study.best_trial)

optimal_regressor.fit(X_train,y_train)

pred_all, pred_std = optimal_regressor.predict(X_scaled, return_std = True)

In [ ]:
R_train_score = optimal_regressor.score(X_train, y_train)
R_test_score = optimal_regressor.score(X_test, y_test)

print("R**2 train : {}".format(R_train_score))
print("R**2 test : {}".format(R_test_score))

In [ ]:

fig, axs = plt.subplots(3,3, figsize = (10,10))
for i, ax in enumerate(axs.ravel()):
    param = settings[i]
    ax.errorbar(data[:,i+1],pred_all, yerr = pred_std, 
                label = 'predicted', linestyle='None', marker='*')
    ax.plot(data[:,i+1],data[:,0], 'r*', label = 'true',
           alpha = 0.1)
    ax.legend()
    ax.set_title('density vs {}'.format(param))
plt.show()

In [ ]:
def gp_to_maximize( regressor, scaler):
    
    def objective(trial):
        x = [[
            trial.suggest_uniform('layer_thickness', 100, 250),
            trial.suggest_uniform('oscillator', 1000, 2500),
            trial.suggest_uniform('recoat speed', 30, 100),
            trial.suggest_uniform('roller speed', 3, 30),
            trial.suggest_uniform('roller rpm', 80, 300),
            trial.suggest_uniform('drying speed', 5, 40),
            trial.suggest_uniform('drying powder', 20, 100),
            trial.suggest_uniform('saturation', 50,100),
            trial.suggest_uniform('bst', 0, 15)
        ]]
        
        x_scaled = scaler.transform(x)
        
        pred, std = regressor.predict(x_scaled, return_std = True)
        
        return pred
    return objective


obj_func = gp_to_maximize(optimal_regressor, x_min_max_scaler)

In [ ]:
study_probe = optuna.create_study(study_name='probe gp', direction='maximize', 
                            storage = 'sqlite:///data/optuna.db',
                           load_if_exists=True)

study_probe.optimize(obj_func, n_trials=10)

In [ ]:
study_probe.best_trial

In [ ]:
test = [[50.0, 2100.0, 57.0, 3.5, 300.0, 6.2, 33, 92.0, 8.2]]
print(optimal_regressor.predict(test, return_std = True))